In [10]:
import pandas as pd
import geopandas as geopd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
pd.set_option('display.max_columns', None)
import ipynb
%run helper.ipynb
%run domain_helper.ipynb

In [11]:
dfb = pd.read_csv('../dataset/Mexico/bc_data/BDmortalidadCAMAbase.csv')
dfb = process_df(dfb,2)

In [12]:
dfb['year'] = dfb['year_of_occurrence']
dfb['age_group'] = dfb['age_group'].apply(update_age_group)
dfb = dfb[ (dfb['population']>0) & (dfb['year']>2008) ]

In [13]:
dfb['ndeaths'] = 1
dfb2 = dfb.groupby(['state_code','municipality_code','age_group','year','location_type']).agg({
    'ndeaths': 'sum',
}).reset_index()

In [14]:
%run domain_helper.ipynb
df = pd.merge(dfb, dfb2, on=['state_code','municipality_code','age_group','year','location_type'])
df['count'] = 1
display(quantl(df, 'count', 'population'))
df = crude_mortality(df)
df = standard_mortality(df, [], [])
df = standard_mortality(df, [], ['state_code','municipality_code'])
df = standard_mortality(df, [], ['state_code','municipality_code','year'])
df = standard_mortality(df, ['age_group'], [])
df = standard_mortality(df, ['age_group'], ['year'])
df = standard_mortality(df, ['age_group'], ['state_code','municipality_code'])
df = standard_mortality(df, ['location_type'], ['state_code','municipality_code'])
df = standard_mortality(df, ['location_type'], [])
df = standard_mortality(df, ['location_type'], ['year'])
df = standard_mortality(df, ['age_group', 'location_type'], [])
df = standard_mortality(df, ['age_group', 'location_type'], ['year'])
df = standard_mortality(df, ['age_group', 'location_type'], ['state_code','municipality_code'])
# display(df.tail(50))

,Quantile_0.0,Quantile_10.0,Quantile_20.0,Quantile_30.0,Quantile_40.0,Quantile_50.0,Quantile_60.0,Quantile_70.0,Quantile_80.0,Quantile_90.0,Quantile_100.0
count_buckets,,,,,,,,,,,
1,1.0,133.0,329.0,737.0,1734.2,3934.0,7522.0,12065.2,18226.0,29529.0,82205.0


In [15]:
display(df.shape)
display(dfb.shape)
display(dfb2.shape)

(87269, 45)

(87269, 28)

(45743, 6)

In [23]:
%run domain_helper.ipynb
ccol = ['year']
dfy = get_confound(df, [], ccol, mmode=0)
dft = get_confound(df, ['age_group'], ccol, mmode=0)
dfy = pd.merge(dfy, dft, on=ccol)
dft = get_confound(df, ['location_type'], ccol, mmode=0)
dfy = pd.merge(dfy, dft, on=ccol)
dft = get_confound(df, ['age_group','location_type'], ccol, mmode=0)
dfy = pd.merge(dfy, dft, on=ccol)
dfy.rename(columns={'smr':'smr_crude'}, inplace=True)
display('Testing SMR across yaers')
smr_columns = [col for col in dfy.columns if col.startswith('smr')]
display(t_test(dfy, smr_columns))

'Testing SMR across yaers'

,col1,col2,t_statistic,p_value,effect_size
0,smr_crude,smr_age_group,-1.050305,0.312711,-0.280706
1,smr_crude,smr_location_type,0.141560,0.889598,0.037834
2,smr_crude,smr_age_group_location_type,-1.027280,0.323024,-0.274552
3,smr_age_group,smr_location_type,1.077381,0.300897,0.287942
4,smr_age_group,smr_age_group_location_type,-0.004253,0.996671,-0.001137
5,smr_location_type,smr_age_group_location_type,-1.064870,0.306314,-0.284598


In [24]:
%run domain_helper.ipynb
ccol = ['state_code','municipality_code']
dfy = get_confound(df, [], ccol, mmode=0)
dft = get_confound(df, ['age_group'], ccol, mmode=0)
dfy = pd.merge(dfy, dft, on=ccol)
dft = get_confound(df, ['location_type'], ccol, mmode=0)
dfy = pd.merge(dfy, dft, on=ccol)
# dft = get_confound(df, ['age_group','location_type','year'], ccol, mmode=0)
# dfy = pd.merge(dfy, dft, on=ccol)
dft = get_confound(df, ['age_group','location_type'], ccol, mmode=0)
dfy = pd.merge(dfy, dft, on=ccol)
# display(dfy)
dfy.rename(columns={'smr':'smr_crude'}, inplace=True)
smr_columns = [col for col in dfy.columns if col.startswith('smr')]
display('Testing SMR across municipalities')
display(t_test(dfy, smr_columns))

'Testing SMR across municipalities'

,col1,col2,t_statistic,p_value,effect_size
0,smr_crude,smr_age_group,3.239124,1.217759e-03,0.070802
1,smr_crude,smr_location_type,14.194661,9.902139e-44,0.310270
2,smr_crude,smr_age_group_location_type,13.144531,5.603501e-38,0.287316
3,smr_age_group,smr_location_type,30.089563,1.368991e-165,0.657705
4,smr_age_group,smr_age_group_location_type,31.122131,4.433593e-175,0.680275
5,smr_location_type,smr_age_group_location_type,5.153172,2.801749e-07,0.112639
